In [ ]:
import pandas as pd

In [4]:
# Define the path to your Excel file
file_path = r'Data\new data.xlsx'

# Load the Excel file
excel_data = pd.ExcelFile(file_path)

# Initialize a dictionary to hold DataFrames
dfs = {}

# Loop through each sheet and store the DataFrame in the dictionary
for sheet_name in excel_data.sheet_names:
    dfs[sheet_name] = pd.read_excel(excel_data, sheet_name=sheet_name, header=0)

# Display the DataFrames
for sheet, df in dfs.items():
    print(f"Sheet name: {sheet}")
    print(df.head())  # Display the first few rows of each DataFrame

Sheet name: Linfield
Empty DataFrame
Columns: [https://mahaffey2024.calicotab.com/mahaffey2024/]
Index: []
Sheet name: YODL 2
     Rk            team            name      category institution  Rk1  R1  \
0    8=  Air Force GoHo     Nathan Good          Open   Air Force  1st  79   
1    15  Air Force GoHo     Grace Hoyte          Open   Air Force  1st  78   
2   96=  Air Force HaMa     Noah Marino  Open, Novice   Air Force  4th  73   
3  114=  Air Force HaMa   Calin Hartzos  Open, Novice   Air Force  4th  74   
4   40=  Air Force KeNe  Audrey Neukomm          Open   Air Force  3rd  74   

   Rk2  R2  Rk3  R3  Rk4  R4  Team  Total    Avg  Trim  Stdev  
0  1st  78  3rd  77  3rd  75     8    309  77.25  77.5   1.48  
1  1st  77  3rd  78  3rd  74     8    307  76.75  77.5   1.64  
2  3rd  76  3rd  75  1st  76     5    300  75.00  75.5   1.22  
3  3rd  75  3rd  74  1st  76     5    299  74.75  74.5   0.83  
4  2nd  78  1st  74  1st  78     9    304  76.00  76.0   2.00  
Sheet name: UCLA IV
 